In [ ]:
!pip install -q biopython

     |████████████████████████████████| 2.3MB 3.6MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = '/content/drive/My Drive/data/ChIP-seq/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp -r '/content/drive/My Drive/dna_nn' .
!ls dna_nn

download.py  load.py


In [ ]:
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.data import Dataset

from dna_nn.load import encode, encoded_shape, gen_from_fasta

%load_ext autoreload
%autoreload 2

# Conv2d

In [ ]:
# generators
word_size = 3
region_size = 0
subset_size = 690 * 139

def cut(x):
    return x[:60]

x_shape = encoded_shape(range(101), word_size, region_size)
encode_func = encode(word_size, region_size)

x_shape = (60 - word_size + 1, ) + x_shape[1:]

train_file = DATA_DIR + 'motif_discovery_train.fasta'
test_file = DATA_DIR + 'motif_discovery_test.fasta'
train_gen = gen_from_fasta(train_file, [cut, encode_func])
test_gen = gen_from_fasta(test_file, [cut, encode_func])

# datasets
bacth_size = 512
prefetch = tf.data.experimental.AUTOTUNE

output_shapes = (x_shape, ())
output_types = (tf.float32, tf.float32)

train_ds = Dataset.from_generator(train_gen, output_types, output_shapes)
# takes about 30 seconds to skip the training data
val_ds = train_ds.skip(subset_size).take(690 * 10)
train_ds = train_ds.take(subset_size).shuffle(500).batch(bacth_size).prefetch(prefetch)

test_ds = Dataset.from_generator(test_gen, output_types, output_shapes)
test_ds = test_ds.take(subset_size).batch(bacth_size).prefetch(prefetch)

print('x shape:', train_ds.element_spec[0].shape)
print('y shape:', train_ds.element_spec[1].shape)

x shape: (None, 58, 64, 1)
y shape: (None,)


In [ ]:
%%time
x_val, y_val = [], []
for d in val_ds:
    x_val.append(d[0])
    y_val.append(d[1])
x_val = tf.convert_to_tensor(x_val)
y_val = tf.convert_to_tensor(y_val)
validation_data = (x_val, y_val)

CPU times: user 47.3 s, sys: 4.03 s, total: 51.3 s
Wall time: 40.9 s


In [ ]:
keras.backend.clear_session()

model = keras.models.load_model('/content/drive/My Drive/dna-nn/UCI/splice-2D.h5')
# model.trainable = False
model.pop()
model.add(keras.layers.Dropout(0.5, name='dropout_2'))
model.add(keras.layers.Dense(2, activation='softmax', name='dense_2'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 56, 62, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 31, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 31, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 55552)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 55552)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               7110784   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0

In [ ]:
# history = model.fit(train_ds, validation_data=validation_data, epochs=5)
# model.trainable = True
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_ds, validation_data=validation_data, epochs=15)

Epoch 1/15
188/188 [==============================] - 44s 235ms/step - loss: 0.7872 - accuracy: 0.5250 - val_loss: 0.6867 - val_accuracy: 0.5539
Epoch 2/15
188/188 [==============================] - 44s 235ms/step - loss: 0.6824 - accuracy: 0.5629 - val_loss: 0.6796 - val_accuracy: 0.5754
Epoch 3/15
188/188 [==============================] - 44s 236ms/step - loss: 0.6730 - accuracy: 0.5804 - val_loss: 0.6749 - val_accuracy: 0.5830
Epoch 4/15
188/188 [==============================] - 44s 236ms/step - loss: 0.6645 - accuracy: 0.5952 - val_loss: 0.6718 - val_accuracy: 0.5859
Epoch 5/15
188/188 [==============================] - 44s 236ms/step - loss: 0.6554 - accuracy: 0.6084 - val_loss: 0.6693 - val_accuracy: 0.5884
Epoch 6/15
188/188 [==============================] - 45s 238ms/step - loss: 0.6481 - accuracy: 0.6203 - val_loss: 0.6676 - val_accuracy: 0.5880
Epoch 7/15
188/188 [==============================] - 47s 248ms/step - loss: 0.6386 - accuracy: 0.6325 - val_loss: 0.6661 - val_ac

In [ ]:
model.evaluate(test_ds)

188/188 [==============================] - 43s 231ms/step - loss: 0.6759 - accuracy: 0.5938


[0.6759299635887146, 0.5937753915786743]